In [109]:
# Thêm autoreload vào để tự động reload lại module nếu có thay đổi code trong module
%load_ext autoreload
%autoreload 2

import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import warnings

# Thêm đường dẫn dẫn đến thư mục cha
sys.path.append(os.path.abspath(".."))
warnings.filterwarnings("ignore")

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
image_path = "./data/test/9.jpg"

In [111]:
def define_original_size(image_path: str) -> tuple[float, float]:
    image = cv2.imread(image_path)
    return image.shape[1], image.shape[0]

In [112]:
origin_size = define_original_size(image_path)

In [113]:
def draw_landmarks(mp_drawing, mp_pose, image : cv2, pose_landmarks):
    """
    Vẽ landmarks lên ảnh
    """
    mp_drawing.draw_landmarks(
        image,
        pose_landmarks,
        mp_pose.POSE_CONNECTIONS,   
        landmark_drawing_spec=mp_drawing.DrawingSpec(
            color=(255, 0, 0),  # Màu sắc của các landmark
            thickness=2,  # Độ dày của các đường nối landmark
            circle_radius=2,  # Bán kính của các điểm landmark
        ),
        connection_drawing_spec=mp_drawing.DrawingSpec(
            color=(0, 255, 0),  # Màu sắc của các đường nối
            thickness=2,  # Độ dày của các đường nối
        ),
    )

    return image

In [114]:
def extract_key_points_from_image(image_path: str) -> np.array:
    # Khởi tạo MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=True, model_complexity=1, smooth_landmarks=True
    )

    image = cv2.imread(image_path)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Phát hiện pose
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        return results.pose_landmarks
    else:
        print(f"Cannot detect pose in image {image_path}")

    pose.close()

    return None

In [115]:
result = extract_key_points_from_image(image_path)

nose = result.landmark[0]

In [116]:
nose

x: 0.163401842
y: 0.479045957
z: -0.598558903
visibility: 0.992158651

# Main handle

In [117]:
def calculate_phase_difference(OA: tuple, OB: tuple) -> float:
    """
    Calculate the phase difference (in degrees) of vector OB relative to vector OA.
    :param OA: A tuple representing the first vector (OA).
    :param OB: A tuple representing the second vector (OB).
    :return: Phase difference in degrees from 0 to 360.
    """
    # Calculate the dot product and magnitudes
    dot_product = OA[0] * OB[0] + OA[1] * OB[1]
    norm_OA = np.sqrt(OA[0] ** 2 + OA[1] ** 2)
    norm_OB = np.sqrt(OB[0] ** 2 + OB[1] ** 2)

    # Calculate the cosine of the angle
    cos_theta = dot_product / (norm_OA * norm_OB)
    
    # Ensure the value is within the valid range for arccos due to floating-point errors
    cos_theta = np.clip(cos_theta, -1, 1)
    
    # Calculate the angle in radians
    theta = np.arccos(cos_theta)
    
    # Calculate the cross product (only the z-component for 2D vectors)
    cross_product = OA[0] * OB[1] - OA[1] * OB[0]
    
    # Adjust the angle based on the direction
    if cross_product < 0:
        theta = 2 * np.pi - theta

    # Convert the angle to degrees
    theta_degrees = np.degrees(theta)

    return theta_degrees

In [118]:
def define_angle(OA: tuple,OB : tuple) -> float:
    dot_product = OA[0] * OB[0] + OA[1] * OB[1]
    norm_OA = np.sqrt(OA[0] ** 2 + OA[1] ** 2)
    norm_OB = np.sqrt(OB[0] ** 2 + OB[1] ** 2)

    cos_theta = dot_product / (norm_OA * norm_OB)
    theta = np.arccos(cos_theta)

    return (theta)

In [119]:
def rotate_point(point: tuple, center: tuple, angle: float, origin_size = (612, 408)) -> tuple:
    x, y = point
    cx, cy = center

    x_new = (x - cx) * np.cos(np.radians(angle)) * origin_size[0] - (y - cy) * np.sin(np.radians(angle)) * origin_size[1] + cx * origin_size[0]
    y_new = (x - cx) * np.sin(np.radians(angle)) * origin_size[0] + (y - cy) * np.cos(np.radians(angle)) * origin_size[1] + cy * origin_size[1]

    return x_new / origin_size[0], y_new / origin_size[1]

In [120]:
def rotate_keypoints(keypoints, image_path: str, origin_size = (612, 408)) -> np.array:
    image = cv2.imread(image_path)
    if keypoints:
            draw_landmarks(mp_drawing, mp_pose, image, keypoints)
            last_dot_index = image_path.rfind(".") # Tìm vị trí cuối cùng của dấu chấm
            image_path_output = image_path[:last_dot_index] + "_output" + image_path[last_dot_index:] # Đặt tên cho ảnh output
            cv2.imwrite(image_path_output, image) # Lưu ảnh đã vẽ landmark

    left_shoulder = keypoints.landmark[11]
    right_shoulder = keypoints.landmark[12]
    left_hip = keypoints.landmark[23]
    right_hip = keypoints.landmark[24]

    center_shoulder = (left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2
    center_hip = (left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2

    center = (center_shoulder[0] + center_hip[0]) / 2, (center_shoulder[1] + center_hip[1]) / 2

    O_center_shoulder = (center_shoulder[0] - center[0], center_shoulder[1] - center[1])
    Oy = (0, -1)

    theta = calculate_phase_difference(O_center_shoulder, Oy)

    print(f"Angle: {theta}")

    # rotate each key point
    for point in keypoints.landmark:
        point_rotated = rotate_point((point.x, point.y), center, theta, origin_size)
        point.x = point_rotated[0]
        point.y = point_rotated[1]

    return keypoints

In [121]:
# draw rotated landmarks
image = cv2.imread(image_path)

image_rotated = draw_landmarks(mp_drawing, mp_pose, image, rotate_keypoints(extract_key_points_from_image(image_path), image_path, origin_size))

# save image
last_dot_index = image_path.rfind(".") # Tìm vị trí cuối cùng của dấu chấm
image_path_output = image_path[:last_dot_index] + "_rotated" + image_path[last_dot_index:] # Đặt tên cho ảnh output
cv2.imwrite(image_path_output, image_rotated) # Lưu ảnh đã vẽ landmark

Angle: 83.53052472193728


True

In [122]:
import numpy as np

def calculate_phase_difference(OA: tuple, OB: tuple) -> float:
    """
    Calculate the phase difference (in degrees) of vector OB relative to vector OA.
    :param OA: A tuple representing the first vector (OA).
    :param OB: A tuple representing the second vector (OB).
    :return: Phase difference in degrees from 0 to 360.
    """
    # Calculate the dot product and magnitudes
    dot_product = OA[0] * OB[0] + OA[1] * OB[1]
    norm_OA = np.sqrt(OA[0] ** 2 + OA[1] ** 2)
    norm_OB = np.sqrt(OB[0] ** 2 + OB[1] ** 2)

    # Calculate the cosine of the angle
    cos_theta = dot_product / (norm_OA * norm_OB)
    
    # Ensure the value is within the valid range for arccos due to floating-point errors
    cos_theta = np.clip(cos_theta, -1, 1)
    
    # Calculate the angle in radians
    theta = np.arccos(cos_theta)
    
    # Calculate the cross product (only the z-component for 2D vectors)
    cross_product = OA[0] * OB[1] - OA[1] * OB[0]
    
    # Adjust the angle based on the direction
    if cross_product < 0:
        theta = 2 * np.pi - theta

    # Convert the angle to degrees
    theta_degrees = np.degrees(theta)

    return theta_degrees

# Define the vectors
OA = (0, -1)
OB = (1, 0)  # Example vector

# Calculate the phase difference in degrees
phase_difference = calculate_phase_difference(OA, OB)
phase_difference


90.0